In [ ]:
from bs4 import BeautifulSoup
import requests
import requests.exceptions
from urllib.parse import urlsplit
from urllib.parse import urlparse
from collections import deque
import sqlite3
import wikipedia

In [32]:
starting_point = "Lists_of_companies"

In [72]:
def create_database(starting_point):
    conn = sqlite3.connect('abbmatch.db')

    c = conn.cursor()

    # Create tables
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS link_map (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, 
            parent_id INTEGER, link TEXT UNIQUE
        )
        """)

    # Create table
    c.execute(
        """
        CREATE TABLE IF NOT EXISTS content (
            id_link INTEGER, content TEXT, 
            title TEXT
        )
        """)

    # Insert the starting point
    c.execute(f"INSERT OR IGNORE INTO link_map VALUES ('1', '-1', '{starting_point}')")

    # Save (commit) the changes
    conn.commit()
    conn.close()
    
def store_links(parent_id, link_list):
    sql = F"INSERT INTO link_map (parent_id, link) VALUES ({parent_id}, ?)"
    
    conn = sqlite3.connect('abbmatch.db')
    c = conn.cursor()
    c.executemany(sql, [(l,) for l in link_list])
    # Save (commit) the changes
    conn.commit()
    conn.close()
    
def _fetchall(sql):
    conn = sqlite3.connect('abbmatch.db')
    c = conn.cursor()
    c.execute(sql)
    res = c.fetchall()
    # Save (commit) the changes
    conn.commit()
    conn.close()
    return res
    
def select(col, table, where=None, limit=10):
    col = col if isinstance(col, str) else ','.join(col)
    where = F"where {where}" if where is not None else ""
    sql = F"SELECT {col} FROM {table} {where} limit {limit}"
    return _fetchall(sql)

def get_unvisited_links(limit=30):
    sql = F"""
        SELECT id, link FROM link_map as A
        LEFT JOIN content as B ON A.id=B.id_link
        WHERE B.content is NULL ORDER BY ID
        LIMIT {limit}
    """
    return _fetchall(sql)
    
create_database(starting_point)

In [71]:
unvisited

[(1, 'Lists_of_companies'),
 (2, 'Beer and breweries by region'),
 (3, 'Big Five law firms'),
 (4, 'Cymbal manufacturers'),
 (5, 'Forbes Global 2000'),
 (6, 'Fortune Global 500'),
 (7, 'List of 100 largest law firms by revenue'),
 (8, 'List of Arab companies'),
 (9, 'List of CAx companies'),
 (10, 'List of CIGS companies'),
 (11, 'List of Canadian mobile phone companies'),
 (12, 'List of Caribbean companies'),
 (13, 'List of Champagne houses'),
 (14, 'List of EDA companies'),
 (15, 'List of European power companies by carbon intensity'),
 (16, 'List of Evolution-Data Optimized network equipment suppliers'),
 (17, 'List of Fibre Channel switches'),
 (18, 'List of French electric utilities'),
 (19, 'List of OGL publishers'),
 (20, 'List of S&P 400 companies'),
 (21, 'List of S&P 500 companies'),
 (22, 'List of SRI International spin-offs'),
 (23, 'List of Six Sigma companies'),
 (24, 'List of United States wireless communications service providers'),
 (25, 'List of advertising technology

In [57]:
store_links(1, wiki_obj.links)

In [73]:
unvisited = get_unvisited_links()

In [75]:
for id_, link in unvisited:
    # Get the wiki object for the link
    wiki_obj = wikipedia.WikipediaPage('Lists_of_companies')
    id_, link

1 Lists_of_companies
